In [13]:
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate
import pandas as pd

def movie_table(url):
    webpage = url
    response = requests.get(webpage)
 
    page = response.text

    soup = BeautifulSoup(page, 'html.parser')
    beautiful_page = soup.get_text()

    ratings = []
    years = []
    titles = []
    movie_nums = []

    lines = beautiful_page.splitlines()

    for line in lines:
        if "(20" in line and ")" in line or '(19' in line and ')' in line:
            years.append(line)

    rep_years = []
    nrep_years = []
    year_indices = []

    for year in years:
        
        if lines.count(year) > 1:
            if year not in rep_years:
                rep_years.append(year)
        if lines.count(year) == 1:
            nrep_years.append(year)

    for i in range(len(rep_years)):
        first_instance = lines.index(rep_years[i])
        year_indices.append(first_instance)

        for j, item in enumerate(lines[first_instance+1:], start=first_instance+1):
            if item == rep_years[i]:
                year_indices.append(j)

    for year in nrep_years:
        if year in lines:
            index = lines.index(year)
            year_indices.append(index)


    for index in sorted(year_indices):
        title = lines[index-1]
        titles.append(title)

        rating = lines[index+6]
        ratings.append(rating)

        movie_num = lines[index-3]
        movie_nums.append(movie_num) 
        

    ratings = [rating.lstrip() for rating in ratings]
    for i in range(len(ratings)):
        if ratings[i] == '':
            ratings[i] = "None"


    for i in range(len(years)):
        for j in range(len(years[i])):
            if years[i][j].isnumeric():
                number_index = years[i].find(years[i][j])
                years[i] = years[i][number_index:number_index+4]
                break


    table_data = list(zip(movie_nums, titles, years, ratings))

    return table_data
    

df_animation_1 = pd.DataFrame(movie_table("https://www.imdb.com/search/title/?title_type=feature&genres=animation&view=simple&ref_=adv_prv"), 
                            columns=['Index', 'Title', 'Year', 'Rating']).assign(Genre='Animation')
df_animation_2 = pd.DataFrame(movie_table("https://www.imdb.com/search/title/?title_type=feature&genres=animation&view=simple&start=51&ref_=adv_nxt"), 
                            columns=['Index', 'Title', 'Year', 'Rating']).assign(Genre='Animation')

df_animation = pd.concat([df_animation_1, df_animation_2], ignore_index=True)
df_animation

,Index,Title,Year,Rating,Genre
0,1.,The Super Mario Bros. Movie,2023,7.3,Animation
1,2.,Suzume,2022,7.8,Animation
2,3.,Puss in Boots: The Last Wish,2022,7.9,Animation
3,4.,Spider-Man: Across the Spider-Verse,2023,None,Animation
4,5.,Your Name.,2016,8.4,Animation
...,...,...,...,...,...
95,96.,Perfect Blue,1997,8,Animation
96,97.,The Good Dinosaur,2015,6.7,Animation
97,98.,Enchanted,2007,7.1,Animation
98,99.,Akira,1988,8,Animation
